In [1]:
from marubatsu import Marubatsu
import numpy as np

board = np.full((3, 3), Marubatsu.EMPTY)
print(board)

[['.' '.' '.']
 ['.' '.' '.']
 ['.' '.' '.']]


In [2]:
%timeit np.array([[Marubatsu.EMPTY] * 3 for y in range(3)])
%timeit np.full((3, 3), Marubatsu.EMPTY)

2.71 μs ± 28.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
1.39 μs ± 31 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [3]:
print(np.zeros((3, 3)))

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [4]:
from marubatsu import NpBoard

def __init__(self, board_size=3, count_linemark=False):
    self.BOARD_SIZE = board_size
    self.count_linemark = count_linemark
    self.board = np.full((self.BOARD_SIZE, self.BOARD_SIZE), Marubatsu.EMPTY)
    if self.count_linemark:
        self.rowcount = {
            Marubatsu.CIRCLE: [0] * self.BOARD_SIZE,
            Marubatsu.CROSS: [0] * self.BOARD_SIZE,
        }
        self.colcount = {
            Marubatsu.CIRCLE: [0] * self.BOARD_SIZE,
            Marubatsu.CROSS: [0] * self.BOARD_SIZE,
        }
        self.diacount = {
            Marubatsu.CIRCLE: [0] * 2,
            Marubatsu.CROSS: [0] * 2,
        }
        
NpBoard.__init__ = __init__

In [5]:
from util import benchmark

boardclass = NpBoard
for count_linemark in [False, True]:
    print(f"boardclass: {boardclass.__name__}, count_linemark {count_linemark}")
    benchmark(mbparams={"boardclass": boardclass, "count_linemark": count_linemark})
    print()

boardclass: NpBoard, count_linemark False
ai2 VS ai2


100%|██████████| 50000/50000 [00:08<00:00, 6050.94it/s]


count     win    lose    draw
o       29454   14352    6194
x       14208   29592    6200
total   43662   43944   12394

ratio     win    lose    draw
o       58.9%   28.7%   12.4%
x       28.4%   59.2%   12.4%
total   43.7%   43.9%   12.4%

ai14s VS ai2


100%|██████████| 50000/50000 [01:18<00:00, 639.81it/s]


count     win    lose    draw
o       49446       0     554
x       44043       0    5957
total   93489       0    6511

ratio     win    lose    draw
o       98.9%    0.0%    1.1%
x       88.1%    0.0%   11.9%
total   93.5%    0.0%    6.5%

ai_abs_dls
 43.9 ms ±   3.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

boardclass: NpBoard, count_linemark True
ai2 VS ai2


100%|██████████| 50000/50000 [00:07<00:00, 7087.24it/s]


count     win    lose    draw
o       29454   14352    6194
x       14208   29592    6200
total   43662   43944   12394

ratio     win    lose    draw
o       58.9%   28.7%   12.4%
x       28.4%   59.2%   12.4%
total   43.7%   43.9%   12.4%

ai14s VS ai2


100%|██████████| 50000/50000 [00:32<00:00, 1540.64it/s]


count     win    lose    draw
o       49446       0     554
x       44043       0    5957
total   93489       0    6511

ratio     win    lose    draw
o       98.9%    0.0%    1.1%
x       88.1%    0.0%   11.9%
total   93.5%    0.0%    6.5%

ai_abs_dls
 43.2 ms ±   3.0 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)



In [6]:
print(board[(1, 0)])
board[(1, 0)]= Marubatsu.CIRCLE
print(board[1, 0])
print(board)

.
o
[['.' '.' '.']
 ['o' '.' '.']
 ['.' '.' '.']]


In [7]:
%timeit board[1][0]
%timeit board[1, 0]
%timeit board[1][0] = Marubatsu.CIRCLE
%timeit board[1, 0] = Marubatsu.CIRCLE

342 ns ± 12.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
229 ns ± 2.34 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
255 ns ± 2.07 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
148 ns ± 2.59 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [8]:
from marubatsu import Markpat
from collections import defaultdict

def getmark_by_move(self, move):
    return self.board[move]    

NpBoard.getmark_by_move = getmark_by_move

def getmark(self, x, y):
    return self.board[x, y]

NpBoard.getmark = getmark

def setmark_by_move(self, move, mark):
    if self.count_linemark:
        x, y = move
        if mark != Marubatsu.EMPTY:
            diff = 1
            changedmark = mark
        else:
            diff = -1
            changedmark = self.board[move]
        self.colcount[changedmark][x] += diff
        self.rowcount[changedmark][y] += diff
        if x == y:
            self.diacount[changedmark][0] += diff
        if x + y == self.BOARD_SIZE - 1:
            self.diacount[changedmark][1] += diff
    self.board[move] = mark
    
NpBoard.setmark_by_move = setmark_by_move

def calc_legal_moves(self):
    legal_moves = [(x, y) for y in range(self.BOARD_SIZE) 
                        for x in range(self.BOARD_SIZE)
                        if self.board[x, y] == Marubatsu.EMPTY]
    return legal_moves    
    
NpBoard.calc_legal_moves = calc_legal_moves

def is_same(self, mark, x, y, dx, dy):
    text_list = [self.board[x + i * dx, y + i * dy] 
                    for i in range(self.BOARD_SIZE)]
    line_text = "".join(text_list)
    return line_text == mark * self.BOARD_SIZE
    
NpBoard.is_same = is_same  

def count_marks(self, turn, last_turn, x, y, dx, dy, datatype):
    count = defaultdict(int)
    for _ in range(self.BOARD_SIZE):
        count[self.board[x, y]] += 1
        x += dx
        y += dy

    if datatype == "dict":
        return count
    else:
        return Markpat(count[last_turn], count[turn], count[Marubatsu.EMPTY])
    
NpBoard.count_marks = count_marks  

In [9]:
for count_linemark in [False, True]:
    print(f"boardclass: {boardclass.__name__}, count_linemark {count_linemark}")
    benchmark(mbparams={"boardclass": boardclass, "count_linemark": count_linemark})
    print()

boardclass: NpBoard, count_linemark False
ai2 VS ai2


100%|██████████| 50000/50000 [00:06<00:00, 7252.75it/s]


count     win    lose    draw
o       29454   14352    6194
x       14208   29592    6200
total   43662   43944   12394

ratio     win    lose    draw
o       58.9%   28.7%   12.4%
x       28.4%   59.2%   12.4%
total   43.7%   43.9%   12.4%

ai14s VS ai2


100%|██████████| 50000/50000 [01:08<00:00, 726.09it/s]


count     win    lose    draw
o       49446       0     554
x       44043       0    5957
total   93489       0    6511

ratio     win    lose    draw
o       98.9%    0.0%    1.1%
x       88.1%    0.0%   11.9%
total   93.5%    0.0%    6.5%

ai_abs_dls
 39.5 ms ±   2.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

boardclass: NpBoard, count_linemark True
ai2 VS ai2


100%|██████████| 50000/50000 [00:06<00:00, 8313.93it/s]


count     win    lose    draw
o       29454   14352    6194
x       14208   29592    6200
total   43662   43944   12394

ratio     win    lose    draw
o       58.9%   28.7%   12.4%
x       28.4%   59.2%   12.4%
total   43.7%   43.9%   12.4%

ai14s VS ai2


100%|██████████| 50000/50000 [00:30<00:00, 1657.19it/s]


count     win    lose    draw
o       49446       0     554
x       44043       0    5957
total   93489       0    6511

ratio     win    lose    draw
o       98.9%    0.0%    1.1%
x       88.1%    0.0%   11.9%
total   93.5%    0.0%    6.5%

ai_abs_dls
 38.8 ms ±   3.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)



In [10]:
mb = Marubatsu()
mb.cmove(0, 0)
mb.cmove(1, 0)
mb.cmove(0, 1)
print(mb)
print(mb.board.board[0])
print(mb.board.board[1])
print(mb.board.board[2])

Turn x
ox.
O..
...

['o', 'o', '.']
['x', '.', '.']
['.', '.', '.']


In [11]:
print(mb.board.board[0][:])
print(mb.board.board[1][:])
print(mb.board.board[2][:])

['o', 'o', '.']
['x', '.', '.']
['.', '.', '.']


In [12]:
print(mb.board.board[:][0])
print(mb.board.board[:][1])
print(mb.board.board[:][2])

['o', 'o', '.']
['x', '.', '.']
['.', '.', '.']


In [13]:
print(mb.board.board[:])
print(mb.board.board)

[['o', 'o', '.'], ['x', '.', '.'], ['.', '.', '.']]
[['o', 'o', '.'], ['x', '.', '.'], ['.', '.', '.']]


In [14]:
mbnp = Marubatsu(boardclass=NpBoard)
mbnp.cmove(0, 0)
mbnp.cmove(1, 0)
mbnp.cmove(0, 1)
print(mbnp)
print(mbnp.board.board[0])
print(mbnp.board.board[1])
print(mbnp.board.board[2])
print(mbnp.board.board[0][:])
print(mbnp.board.board[1][:])
print(mbnp.board.board[2][:])
print(mbnp.board.board[0, :])
print(mbnp.board.board[1, :])
print(mbnp.board.board[2, :])

Turn x
ox.
O..
...

['o' 'o' '.']
['x' '.' '.']
['.' '.' '.']
['o' 'o' '.']
['x' '.' '.']
['.' '.' '.']
['o' 'o' '.']
['x' '.' '.']
['.' '.' '.']


In [15]:
print(mbnp.board.board[:, 0])
print(mbnp.board.board[:, 1])
print(mbnp.board.board[:, 2])

['o' 'x' '.']
['o' '.' '.']
['.' '.' '.']


In [16]:
print("".join(mbnp.board.board[0, :]))
print("".join(mbnp.board.board[:, 0]))

oo.
ox.


In [17]:
player = Marubatsu.CIRCLE
print("0 列の計算")
%timeit mbnp.board.is_same(player, x=0, y=0, dx=1, dy=0)
%timeit "".join(mbnp.board.board[0, :]) == "ooo"
print("0 行の計算")
%timeit mbnp.board.is_same(player, x=0, y=0, dx=0, dy=1)
%timeit "".join(mbnp.board.board[:, 0]) == player * mb.BOARD_SIZE


0 列の計算
1.32 μs ± 23.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
1.63 μs ± 29.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
0 行の計算
1.3 μs ± 23.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
1.73 μs ± 40.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [18]:
x = 0
y = 0
dx = 1
dy = 0
%timeit [mbnp.board.board[x + i * dx][y + i * dy]  for i in range(mbnp.BOARD_SIZE)]
%timeit mbnp.board.board[0, :]

1.34 μs ± 56.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
190 ns ± 12.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [19]:
print(mbnp.board.board[0, :].tolist())

['o', 'o', '.']


In [20]:
%timeit "".join(mbnp.board.board[0, :].tolist()) == "ooo"

406 ns ± 21.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [33]:
%timeit list(mbnp.board.board[0, :])
%timeit mbnp.board.board[0, :].tolist()

1.47 μs ± 40 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
284 ns ± 5.28 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [21]:
mbnp = Marubatsu(boardclass=NpBoard)
mbnp.cmove(0, 0)
mbnp.cmove(1, 1)
mbnp.cmove(2, 2)
print(mbnp)
print(np.diag(mbnp.board.board))

Turn x
o..
.x.
..O

['o' 'x' 'o']


In [22]:
print(mbnp.board.board)
print(np.fliplr(mbnp.board.board))


[['o' '.' '.']
 ['.' 'x' '.']
 ['.' '.' 'o']]
[['.' '.' 'o']
 ['.' 'x' '.']
 ['o' '.' '.']]


In [23]:
print(np.diag(np.fliplr(mbnp.board.board)))

['.' 'x' '.']


In [24]:
def is_winner(self, player, last_move):
    x, y = last_move
    if self.count_linemark:
        if self.rowcount[player][y] == self.BOARD_SIZE or \
        self.colcount[player][x] == self.BOARD_SIZE:
            return True
        # 左上から右下方向の判定
        if x == y and self.diacount[player][0] == self.BOARD_SIZE:
            return True
        # 右上から左下方向の判定
        if x + y == self.BOARD_SIZE - 1 and \
            self.diacount[player][1] == self.BOARD_SIZE:
            return True
    else:
        winstr = player * self.BOARD_SIZE
        if "".join(self.board[x, :].tolist()) == winstr  or \
        "".join(self.board[:, y].tolist()) == winstr:
            return True
        # 左上から右下方向の判定
        if x == y and "".join(np.diag(self.board).tolist()) == winstr:
            return True
        # 右上から左下方向の判定
        if x + y == self.BOARD_SIZE - 1 and \
            "".join(np.diag(np.fliplr(self.board)).tolist()) == winstr:
            return True
    
    # どの一直線上にも配置されていない場合は、player は勝利していないので False を返す
    return False

NpBoard.is_winner = is_winner

In [25]:
for count_linemark in [False, True]:
    print(f"boardclass: {boardclass.__name__}, count_linemark {count_linemark}")
    benchmark(mbparams={"boardclass": boardclass, "count_linemark": count_linemark})
    print()

boardclass: NpBoard, count_linemark False
ai2 VS ai2


100%|██████████| 50000/50000 [00:06<00:00, 7778.06it/s]


count     win    lose    draw
o       29454   14352    6194
x       14208   29592    6200
total   43662   43944   12394

ratio     win    lose    draw
o       58.9%   28.7%   12.4%
x       28.4%   59.2%   12.4%
total   43.7%   43.9%   12.4%

ai14s VS ai2


100%|██████████| 50000/50000 [01:08<00:00, 732.36it/s]


count     win    lose    draw
o       49446       0     554
x       44043       0    5957
total   93489       0    6511

ratio     win    lose    draw
o       98.9%    0.0%    1.1%
x       88.1%    0.0%   11.9%
total   93.5%    0.0%    6.5%

ai_abs_dls
 39.2 ms ±   5.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

boardclass: NpBoard, count_linemark True
ai2 VS ai2


100%|██████████| 50000/50000 [00:06<00:00, 8005.89it/s]


count     win    lose    draw
o       29454   14352    6194
x       14208   29592    6200
total   43662   43944   12394

ratio     win    lose    draw
o       58.9%   28.7%   12.4%
x       28.4%   59.2%   12.4%
total   43.7%   43.9%   12.4%

ai14s VS ai2


100%|██████████| 50000/50000 [00:30<00:00, 1659.97it/s]


count     win    lose    draw
o       49446       0     554
x       44043       0    5957
total   93489       0    6511

ratio     win    lose    draw
o       98.9%    0.0%    1.1%
x       88.1%    0.0%   11.9%
total   93.5%    0.0%    6.5%

ai_abs_dls
 38.8 ms ±   2.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)



In [26]:
l = [i for i in range(10000)]
print(sum(l))
a = np.array(l)
print(np.sum(a))

49995000
49995000


In [27]:
print("sum(l)")
%timeit sum(l)
print("np.sum(a)")
%timeit np.sum(a)

sum(l)
51.2 μs ± 113 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
np.sum(a)
5.37 μs ± 170 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [28]:
print("np.sum(l)")
%timeit np.sum(l)
print("sum(a)")
%timeit sum(a)

np.sum(l)
411 μs ± 1.84 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
sum(a)
563 μs ± 3.01 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
